# CIFAR10_Test

使用CIFAR10数据集对CNN进行训练及测试

In [13]:
import tensorflow as tf
import numpy as np
import math
#import matplotlib.pyplot as plt
#%matplotlib inline
import time
import os
import tensornets as nets
import cv2
import scipy.io as sio
# for auto-reloading external modules
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [14]:
from cs231n.data_utils import load_CIFAR10
import sys
if sys.platform == "linux" :
    cifar10_dir = "/home/z_tomcato/cs231n/assignment2/assignment2/cs231n/datasets/cifar-10-batches-py"
else:
    cifar10_dir = 'cs231n/datasets'
    
def get_CIFAR10_data(num_training=10000, num_validation=1000, num_test=10000):
    """
    Load the CIFAR-10 dataset from disk and perform preprocessing to prepare
    it for the two-layer neural net classifier. These are the same steps as
    we used for the SVM, but condensed to a single function.  
    """
    # Load the raw CIFAR-10 data
    # cifar10_dir = '../assignment2/cs231n/datasets'
    X_train, y_train, X_test, y_test = load_CIFAR10(cifar10_dir)

    # Subsample the data
    mask = range(num_training, num_training + num_validation)
    X_val = X_train[mask]
    y_val = y_train[mask]
    mask = range(num_training)
    X_train = X_train[mask]
    y_train = y_train[mask]
    mask = range(num_test)
    X_test = X_test[mask]
    y_test = y_test[mask]

    # Normalize the data: subtract the mean image
    mean_image = np.mean(X_train, axis=0)
    X_train -= mean_image
    X_val -= mean_image
    X_test -= mean_image

    return X_train, y_train, X_val, y_val, X_test, y_test


# Invoke the above function to get our data.
X_train, y_train, X_val, y_val, X_test, y_test = get_CIFAR10_data()
print('Train data shape: ', X_train.shape)
print('Train labels shape: ', y_train.shape)
print('Validation data shape: ', X_val.shape)
print('Validation labels shape: ', y_val.shape)
print('Test data shape: ', X_test.shape)
print('Test labels shape: ', y_test.shape)

Train data shape:  (10000, 32, 32, 3)
Train labels shape:  (10000,)
Validation data shape:  (1000, 32, 32, 3)
Validation labels shape:  (1000,)
Test data shape:  (10000, 32, 32, 3)
Test labels shape:  (10000,)


## Tensornets测试

In [15]:
"""
inputs = tf.placeholder(tf.float32, [None, 224, 224, 3])

#model = nets.InceptionResNet2
model = nets.MobileNet100

newModel = model(inputs)
img = nets.utils.load_img('cat.png', target_size=256, crop_size=224)
assert img.shape == (1, 224, 224, 3)
with tf.Session() as sess:
    img = newModel.preprocess(img)  # equivalent to img = nets.preprocess(model, img)
    sess.run(newModel.pretrained())  # equivalent to nets.pretrained(model)
    scores = sess.run(newModel, {inputs: img})
    #rint(preds)
print(nets.utils.decode_predictions(scores, top=1)[0])
#print(tf.argmax(tf.squeeze(preds, [0], axis = 1)))
predictions = tf.argmax(scores, axis = 1)
print(predictions)
[(u'n02124075', u'Egyptian_cat', 0.28067636), (u'n02127052', u'lynx', 0.16826575)]"""

"\ninputs = tf.placeholder(tf.float32, [None, 224, 224, 3])\n\n#model = nets.InceptionResNet2\nmodel = nets.MobileNet100\n\nnewModel = model(inputs)\nimg = nets.utils.load_img('cat.png', target_size=256, crop_size=224)\nassert img.shape == (1, 224, 224, 3)\nwith tf.Session() as sess:\n    img = newModel.preprocess(img)  # equivalent to img = nets.preprocess(model, img)\n    sess.run(newModel.pretrained())  # equivalent to nets.pretrained(model)\n    scores = sess.run(newModel, {inputs: img})\n    #rint(preds)\nprint(nets.utils.decode_predictions(scores, top=1)[0])\n#print(tf.argmax(tf.squeeze(preds, [0], axis = 1)))\npredictions = tf.argmax(scores, axis = 1)\nprint(predictions)\n[(u'n02124075', u'Egyptian_cat', 0.28067636), (u'n02127052', u'lynx', 0.16826575)]"

In [16]:
def load_img_from_tensor(x, target_size=None, crop_size=None, interp=cv2.INTER_CUBIC):

    minSize = min(x.shape[1:3])
    imgs = None
    if target_size:
        if isinstance(target_size, int):
            hw_tuple = (x.shape[1] * target_size // minSize, x.shape[2] * target_size // minSize)
        else:
            hw_tuple = (target_size[1], target_size[0])
        imgs = np.zeros((x.shape[0],hw_tuple[0],hw_tuple[1], 3), dtype=np.uint8)
        if x.shape[1:3] != hw_tuple:
            for i in range(x.shape[0]):
                imgs[i,:, :, :] = cv2.resize(x[i, :, :, :], hw_tuple, interpolation=interp)
    if crop_size is not None:
        imgs = nets.utils.crop(imgs, crop_size)
        
    return imgs

def img_preprocess(x):
    # Copied from keras (equivalent to the same as in TF Slim)
    x = x.copy()
    x = x / 255.
    x = x - 0.5
    x = x * 2.
    return x

## Run Module

In [ ]:

#model = nets.MobileNet100

def run_model(session, Xd, yd, Xv, yv, num_class = 10, epochs=3, batch_size=100,print_every=10, learning_rate = 1e-5, dropout = 0.5):
    print("Batch dataset initialized.\n# of training data: {}\n# of test data: {}\n# of class: {}"
          .format(Xd.shape[0], Xv.shape[0], 10))
    
    # shuffle indicies
    train_indicies = np.arange(Xd.shape[0])
    np.random.shuffle(train_indicies)
        
    with tf.Session() as sess:

        inputs = tf.placeholder(tf.float32, [None, 224, 224, 3])
        outputs = tf.placeholder(tf.int32, [None])
        
        cnn_net = nets.MobileNet100(inputs, is_training = True, classes = num_class)
        
        cnn_loss = tf.losses.softmax_cross_entropy(tf.one_hot(outputs,num_class, dtype=tf.int32), cnn_net)
        cnn_train = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cnn_loss)
        
        sess.run(tf.global_variables_initializer())
        nets.pretrained(cnn_net)        
        
        # tensorboard setting

        fileName = time.strftime("%Y%m%d_%H%M%S", time.localtime())
        fileName = os.path.normcase("./result/" + fileName)
        summary_writer = tf.summary.FileWriter(fileName, sess.graph)
        
        global_step = 0
                  
        for current_epoch in range(epochs):
            # training step
            ###for x_batch, y_batch in batch_set.batches():
            print("#############################Epoch Start##############################")
            
            for i in range(int(math.ceil(Xd.shape[0]/batch_size))):
                start = time.time()
                start_idx = (i*batch_size)%Xd.shape[0]
                idx = np.int32(train_indicies[start_idx:start_idx+batch_size])
                                      
                batch_Xd = load_img_from_tensor(Xd[idx,:, :, :], target_size=256, crop_size=224)
                batch_Xd = cnn_net.preprocess(batch_Xd) 
                batch_yd = yd[idx]
                feed = {inputs : batch_Xd, outputs : batch_yd}                
                
                global_step = global_step + 1
                
                cnn_predictions = tf.argmax(cnn_net, axis = 1)
                cnn_correct_prediction = tf.equal(tf.cast(cnn_predictions, dtype=tf.int32), batch_yd)
                cnn_accuracy = tf.reduce_mean(tf.cast(cnn_correct_prediction, tf.float32))
        
                train_summary = tf.summary.merge([tf.summary.scalar("train_loss", cnn_loss),
                                          tf.summary.scalar("train_accuracy", cnn_accuracy)])

                _, loss, scores,accuracy, summary = sess.run([cnn_train, cnn_loss, 
                                                              cnn_net, cnn_accuracy, train_summary], feed_dict=feed)
                
                summary_writer.add_summary(summary, global_step)

                
                if global_step % print_every == 0:
                    print("{}/{} ({} epochs) step, loss : {:.6f}, accuracy : {:.3f}, time/batch : {:.3f}sec"
                          .format(global_step, int(round(Xd.shape[0]/batch_size)) * epochs, current_epoch,
                                  loss, accuracy, time.time() - start))

            # test step
            start, avg_loss, avg_accuracy = time.time(), 0, 0
            
            test_summary = tf.summary.merge([tf.summary.scalar("val_loss", cnn_loss),
                                             tf.summary.scalar("val_accuracy", cnn_net)])
            
            Xv = cnn_net.preprocess(Xv) 
            feed = {inputs : Xv, outputs : yv}
            loss, accuracy, summary = sess.run([cnn_loss, cnn_accuracy, test_summary], feed_dict=feed)

            summary_writer.add_summary(summary, current_epoch)
            print("{} epochs test result. loss : {:.6f}, accuracy : {:.3f}, time/batch : {:.3f}sec"
                  .format(current_epoch, loss , accuracy , time.time() - start))
            
            print("\n")
    return 
    
   

In [ ]:
def load_wiki(wiki_path, num_training=49000, num_validation=1000, num_test=1000,):
   
    wiki_path = ""
    if sys.platform == "linux" :
            wiki_path = "/devdata/wiki/"
    else:
            wiki_path = "G:\\MachineLearning\\wiki\\wiki\\"

    mat_path = wiki_path + 'wiki_with_age.mat'
    data = sio.loadmat(mat_path) 
    
    img_paths = []
    for i in range(len(wiki_data[6][0])):
        wiki_data = data['wiki'][0][0]
        face_score =wiki_data[6][0][i]
        if face_score != float("-inf"):
            full_path = wiki_path + wiki_data[0][0][2][0][i][0]
            img = cv2.imread(full_path)
            cv2.imshow("test", img)

            face_loc = wiki_data[5][0][i][0]
            print(face_loc)
            face_loc = face_loc.astype("int32")
            print(face_loc)
            roi_img = img[face_loc[1]:face_loc[3], face_loc[0]:face_loc[2]]
            temp_img = cv2.resize(roi_img, ((face_loc[3]-face_loc[1]) * 3, (face_loc[2]-face_loc[0]) * 3))
            cv2.imshow("temp_img", temp_img)
            cv2.imshow("roi_img", roi_img)
            print("age: ", wiki_data[8][0][i])
            gender = wiki_data[3][0][i]
            if gender == 0:
                print("女")
            else:
                print("男")
        else:
            print("没找到人脸")
    

In [ ]:
tf.reset_default_graph()
with tf.Session() as sess:
    #with tf.device("/cpu:0"): #"/cpu:0" or "/gpu:0" 
    #sess.run(tf.global_variables_initializer())
    #print('Training')
    
    run_model(sess,X_train,y_train,X_val,y_val, epochs=4, batch_size=500,print_every=100, learning_rate = 1e-5)

Batch dataset initialized.
# of training data: 10000
# of test data: 1000
# of class: 10
#############################Epoch Start##############################